In [12]:
import pandas as pd

summer = pd.read_csv("../AssB_Input_Group4_summer.csv", index_col = "TimeStamp")
summer.columns

Index(['Start date/time', 'End date/time', 'Residential load [kW]',
       'PV generation [kW]', 'Electricity price [euro/kWh]',
       'Marginal emission factor [kg CO2eq/kWh]'],
      dtype='object')

In [14]:
summer

,Start date/time,End date/time,Residential load [kW],PV generation [kW],Electricity price [euro/kWh],Marginal emission factor [kg CO2eq/kWh]
TimeStamp,,,,,,
0,07/07/2018 00:00,07/07/2018 00:15,0.36722,0.0,0.04771,0.385803
1,07/07/2018 00:15,07/07/2018 00:30,0.34986,0.0,0.04771,0.385803
2,07/07/2018 00:30,07/07/2018 00:45,0.33334,0.0,0.04771,0.385803
3,07/07/2018 00:45,07/07/2018 01:00,0.31794,0.0,0.04771,0.385803
4,07/07/2018 01:00,07/07/2018 01:15,0.30296,0.0,0.04580,0.365847
...,...,...,...,...,...,...
283,09/07/2018 22:45,09/07/2018 23:00,0.42014,0.0,0.05259,0.416062
284,09/07/2018 23:00,09/07/2018 23:15,0.41790,0.0,0.04937,0.955588
285,09/07/2018 23:15,09/07/2018 23:30,0.41048,0.0,0.04937,0.955588


In [30]:
# -*- coding: utf-8 -*-
"""
% Energy in the Built Environment
% Assignment 2: Optimal Home Energy Management
% Dr. Tarek AlSkaif

"""
import gurobipy as gp
import csv
import pandas as pd #for csv reading
import numpy as np 
import matplotlib.pyplot as plt #for plotting

"""
Import your input data for the model
"""
summer = pd.read_csv("../AssB_Input_Group4_summer.csv")


    # dynamic electricity prices vector
    #household's 15-min PV generation vector
    #household's 15-min demand vector

#for now setting up just for summer as thinking when we make it a function can specify summer or winter
Ppv = summer['PV generation [kW]']
Pdem = summer['Residential load [kW]']

"""
Parameters value
"""
######## Time-step
Delta_t = 0.25 # 15 minute (0.25 hour) intervals
T=24*3*1/Delta_t #number of time-slots (in three days)
t=np.linspace(1, T, num=int(T)) 

######## Limits on grid and max, min, and initial SOC
Pgridmax = 3 #[kW]
Pbatmax = 4 #[kW]

SoC_min = 0.2 #[-] (battery min state of charge)
SoC_max = 1 #[-] (battery max state of charge)
SoC0 = 0.5 #[-] (initial battery state of charge at the beginning of the day)

C_bat = 13.5 #battery capacity parameter for a Tesla Powerwall rated at 13,5 [kWh]
eff_dis = 0.94 #battery discharging efficeicny
eff_ch = 0.94 #battery charging efficeicny

######## Plot power demand and PV generation data
f1 = plt.figure(1)

######## other parameters we added, N is number of power sources (note batt_ch and batt_disch as seperate)
N = 3

"""
Step 1: Create a model
"""
m=gp.Model()

"""
Step 2: Define variables
"""
######## Define your decision variables for the time horizon using addVars
T=int(24*3*1/Delta_t)
Pbat_cha = m.addVars(T, lb= 0, ub= float('inf'), obj= 1, vtype= gp.GRB.CONTINUOUS, name= "Pbat_ch")
Pbat_cha 

Pgrid = m.addVars(T, lb= float('-inf'), ub= float('inf'), vtype= gp.GRB.CONTINUOUS, name= "Pgrid")
Pgrid

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>,
 7: <gurobi.Var *Awaiting Model Update*>,
 8: <gurobi.Var *Awaiting Model Update*>,
 9: <gurobi.Var *Awaiting Model Update*>,
 10: <gurobi.Var *Awaiting Model Update*>,
 11: <gurobi.Var *Awaiting Model Update*>,
 12: <gurobi.Var *Awaiting Model Update*>,
 13: <gurobi.Var *Awaiting Model Update*>,
 14: <gurobi.Var *Awaiting Model Update*>,
 15: <gurobi.Var *Awaiting Model Update*>,
 16: <gurobi.Var *Awaiting Model Update*>,
 17: <gurobi.Var *Awaiting Model Update*>,
 18: <gurobi.Var *Awaiting Model Update*>,
 19: <gurobi.Var *Awaiting Model Update*>,
 20: <gurobi.Var *Awaiting Model Update*>,
 21: <gurobi.Var *Awaiting Model Update*>,
 22: <gurobi.Var *Awaiting Model Update*>,
 23: <gurobi.Var *Awa

<Figure size 432x288 with 0 Axes>